In [1]:
import malmo
import malmo.MalmoPython as MalmoPython
import os
import sys
import time
import json
import numpy as np

import gym
from gym import error, spaces, utils
from gym.utils import seeding

In [2]:
import functools
print = functools.partial(print, flush=True)

In [3]:
def Menger(xorg, yorg, zorg, size, blocktype, variant, holetype):
    #draw solid chunk
    genstring = GenCuboidWithVariant(xorg,yorg,zorg,xorg+size-1,yorg+size-1,zorg+size-1,blocktype,variant) + "\n"
    #now remove holes
    unit = size
    while (unit >= 3):
        w=unit//3
        for i in range(0, size, unit):
            for j in range(0, size, unit):
                x=xorg+i
                y=yorg+j
                genstring += GenCuboid(x+w,y+w,zorg,(x+2*w)-1,(y+2*w)-1,zorg+size-1,holetype) + "\n"
                y=yorg+i
                z=zorg+j
                genstring += GenCuboid(xorg,y+w,z+w,xorg+size-1, (y+2*w)-1,(z+2*w)-1,holetype) + "\n"
                genstring += GenCuboid(x+w,yorg,z+w,(x+2*w)-1,yorg+size-1,(z+2*w)-1,holetype) + "\n"
        unit = w
    return genstring

def GenCuboid(x1, y1, z1, x2, y2, z2, blocktype):
    return '<DrawCuboid x1="' + str(x1) + '" y1="' + str(y1) + '" z1="' + str(z1) + '" x2="' + str(x2) + '" y2="' + str(y2) + '" z2="' + str(z2) + '" type="' + blocktype + '"/>'

def GenCuboidWithVariant(x1, y1, z1, x2, y2, z2, blocktype, variant):
    return '<DrawCuboid x1="' + str(x1) + '" y1="' + str(y1) + '" z1="' + str(z1) + '" x2="' + str(x2) + '" y2="' + str(y2) + '" z2="' + str(z2) + '" type="' + blocktype + '" variant="' + variant + '"/>'

In [4]:
drawingDecorator = '''<DrawingDecorator>
                    <DrawSphere x="-27" y="70" z="0" radius="30" type="air"/>''' + Menger(-40, 40, -13, 27, "stone", "smooth_granite", "air") + '''
                    <DrawCuboid x1="-25" y1="39" z1="-2" x2="-29" y2="39" z2="2" type="lava"/>
                    <DrawCuboid x1="-26" y1="39" z1="-1" x2="-28" y2="39" z2="1" type="obsidian"/>
                    <DrawBlock x="-27" y="39" z="0" type="diamond_block"/>
                  </DrawingDecorator>'''

missionXML='''<?xml version="1.0" encoding="UTF-8" standalone="no" ?>
            <Mission xmlns="http://ProjectMalmo.microsoft.com" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance">
            
              <About>
                <Summary>Hello world!</Summary>
              </About>
              
            <ServerSection>
              <ServerInitialConditions>
                <Time>
                    <StartTime>1000</StartTime>
                    <AllowPassageOfTime>false</AllowPassageOfTime>
                </Time>
                <Weather>clear</Weather>
              </ServerInitialConditions>
              <ServerHandlers>
                  <FlatWorldGenerator generatorString="3;7,44*49,73,35:1,159:4,95:13,35:13,159:11,95:10,159:14,159:6,35:6,95:6;12;"/>
                  <DrawingDecorator>
                      <DrawBlock x="0" y="56" z="-1" type="diamond_block" />
                  </DrawingDecorator>
                  <ServerQuitFromTimeUp timeLimitMs="10000"/>
                  <ServerQuitWhenAnyAgentFinishes/>
                </ServerHandlers>
              </ServerSection>
              
              <AgentSection mode="Survival">
                <Name>MalmoTutorialBot</Name>
                <AgentStart>
                    <Placement x="0.5" y="56.0" z="0.5" yaw="180"/>
                </AgentStart>
                <AgentHandlers>
                  <ObservationFromFullStats/>
                  <ObservationFromGrid>
                      <Grid name="floor3x3">
                        <min x="-1" y="-1" z="-1"/>
                        <max x="1" y="0" z="1"/>
                      </Grid>
                  </ObservationFromGrid>
                  <ObservationFromDistance>
                      <Marker name="Origin" x="0.5" y="56.0" z="0.5" />
                  </ObservationFromDistance>
                  <ContinuousMovementCommands turnSpeedDegs="180"/>
                </AgentHandlers>
              </AgentSection>
            </Mission>'''




In [5]:
def setup_malmo() -> [MalmoPython.AgentHost, MalmoPython.MissionSpec, MalmoPython.MissionRecordSpec]:
    # Create default Malmo objects:

    agent_host = MalmoPython.AgentHost()
    try:
        agent_host.parse( sys.argv )
    except RuntimeError as e:
        print('ERROR:',e)
        print(agent_host.getUsage())
        exit(1)
    if agent_host.receivedArgument("help"):
        print(agent_host.getUsage())
        exit(0)

    my_mission = MalmoPython.MissionSpec(missionXML, True)
    my_mission_record = MalmoPython.MissionRecordSpec()

    # Attempt to start a mission:
    max_retries = 3
    for retry in range(max_retries):
        try:
            agent_host.startMission( my_mission, my_mission_record )
            break
        except RuntimeError as e:
            if retry == max_retries - 1:
                print("Error starting mission:",e)
                exit(1)
            else:
                time.sleep(2)

    # Loop until mission starts:
    print("Waiting for the mission to start ", end=' ')
    world_state = agent_host.getWorldState()
    while not world_state.has_mission_begun:
        print(".", end="")
        time.sleep(0.1)
        world_state = agent_host.getWorldState()
        for error in world_state.errors:
            print("Error:",error.text)

    print()
    print("Mission running ", end=' ')


    return agent_host, my_mission, my_mission_record

In [6]:
def try_observation(agent_host):
    world_state = agent_host.getWorldState()
    for error in world_state.errors:
            print("Error:",error.text)
            
    finished = not world_state.is_mission_running
    state = None
    
    if world_state.number_of_observations_since_last_state > 0:
        msg = world_state.observations[-1].text                 # Yes, so get the text
        observations = json.loads(msg)                          # and parse the JSON
        grid = observations.get(u'floor3x3', 0)                 # and get the grid we asked for
        #dist = observations.get(u'distanceFromOrigin', 0)
        #px = observations.get(u'XPos', 0)
        #py = observations.get(u'YPos', 0)
        #print(px, py, pz)
        dist = 0.5 - observations.get(u'ZPos', 0)
        
        state = grid, dist
    
    return finished, state

In [7]:
def get_observation(agent_host):
    done, state = try_observation(agent_host)
    
    while(not done and state is None):
        time.sleep(0.1)
        done, state = try_observation(agent_host)
    
    grid, dist = None, None
    if(state is not None):
        grid, dist = state
    return done, grid, dist

In [8]:
def convert_observation(malmo_obs, output_shape):
    '''
    Returns the observation in the form of a MultiBinary space of the right shape
    1 if block is occupied, 0 if block is empty
    '''
    array = np.array(malmo_obs).reshape(output_shape)
    return np.where(array == 'air', 0, 1)

In [9]:
def handle_action(agent_host, actions):
    #print(actions)
    mov, jmp = actions
    mov -= 1 # between -1 and 1
    
    agent_host.sendCommand("move " + str(mov))
    agent_host.sendCommand("jump " + str(jmp))

In [10]:
class MinecraftEnv(gym.Env):
    
    metadata = {'render.modes': ['human']}

    def __init__(self):        
        # set up environment variables (state space, action space, rewards...)
        self.obs_shape = (2, 3, 3) #[y, z, x]
        self.observation_space = spaces.MultiBinary(self.obs_shape)
        self.action_space = spaces.MultiDiscrete((3, 2)) # [0] => move -1 / 0 / 1 ; [1] => jump 0 / 1
        self.distance = 0
    
    def step(self, action):
        # handle action
        handle_action(self.agent_host, action)
        # get new observation
        finished, grid, dist = get_observation(self.agent_host)
        if finished:
            return None, 0, True, {}
        
        obs = convert_observation(grid, self.obs_shape)
        # calculate reward
        reward = dist - self.distance
        self.distance = dist
        
        return obs, reward, finished, {}
    
    def reset(self):
        # set up malmo
        agent_host, my_mission, my_mission_record = setup_malmo()
        self.agent_host = agent_host
        self.my_mission = my_mission
        self.my_mission_record = my_mission_record
        self.distance = 0
        
        finished, grid, dist = get_observation(self.agent_host)
        return convert_observation(grid, self.obs_shape)
    
    def render(self, mode='human'):
        pass
  
    def close(self):
        pass

In [11]:
env = MinecraftEnv()
for i_episode in range(20):
    observation = env.reset()
    for t in range(100):
        env.render()
        #print(observation)        
        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)
        print(reward)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()


ERROR: Caught std::exception: unrecognised option '-f'

Malmo version: 0.36.0

Allowed options:
  -h [ --help ]         show description of allowed options
  --test                run this as an integration test


Waiting for the mission to start  .....
Mission running  0.0
0.0
0.01197777851484716
0.059963081301078525
0.05376870849902371
0.07429041976412165
0.0
0.0
0.0
-0.03396244448072705
-0.020035085941748076
-0.027428218642424118
-0.0929331473113632
-0.13388030436942444
-0.18573828835466555
-0.3196354090331973
-0.15659772092215896
0.0
-0.03849350038421706
-0.04030678457224823
-0.021755257198939004
-0.107298497375786
-0.0722966907320659
-0.0313952862934983
0.042726539013560494
0.050664632070149374
0.07808725442899345
0.10755111213806545
0.11626477735669782
0.09306491967889996
0.02418879507967442
-0.032050064844837856
-0.05683764120965318
-0.08437441407155366
-0.021135848214332897
-0.025517363251871483
0.006409612958393485
0.010509119036160364
0.009678287940446673
-0.00840517762792592

KeyboardInterrupt: 

In [ ]:
agent_host, my_mission, my_mission_record = setup_malmo()


In [ ]:
a, b, c= get_observation(agent_host)